In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
dataname = '/home/chris/Code/PointClouds/data/ply/Cover/CoverCleaned.ply'
pcd = o3d.io.read_point_cloud(dataname)

In [3]:
pcd_center = pcd.get_center()
pcd.translate(-pcd_center)

In [4]:
nn = 16
std_multiplier = 10

filtered_pcd = pcd.remove_statistical_outlier(nn, std_multiplier)

outliers = pcd.select_by_index(filtered_pcd[1], invert = True)
outliers.paint_uniform_color([1,0,0])
filtered_pcd = filtered_pcd[0]

In [5]:
voxel_size = 0.01

pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size=voxel_size)

In [6]:
nn_distance = np.mean(pcd.compute_nearest_neighbor_distance())
radius_normals = nn_distance*4

pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius_normals, max_nn =16), fast_normal_computation=True)

In [8]:
pt_to_plane_dist = 0.02

plane_model, inliers = pcd.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=3,num_iterations=1000)
[a,b,c,d] = plane_model

print(f'Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0')\

inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)

Plane equation: 0.00x + 0.01y + 1.00z + -476.61 = 0


In [10]:
#Multi-order RANSAC

max_plane_idx = 6
pt_to_plane_dist = 0.02

segment_models = {}
segments = {}
rest = pcd

for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert = True)
    print("pass",i+1,"/",max_plane_idx,"done.")


pass 1 / 6 done.
pass 2 / 6 done.
pass 3 / 6 done.
pass 4 / 6 done.
pass 5 / 6 done.
pass 6 / 6 done.


In [11]:
import plotly.graph_objects as go
ransac_fig = go.Figure()

# Iterate through each segment and add it as a trace to the figure
for i, segment in segments.items():
    # Extract x, y, z coordinates of each segment
    segment_points = np.asarray(segment.points)

    # Extract x, y, z coordinates of each segment
    x = segment_points[:, 0]
    y = segment_points[:, 1]
    z = segment_points[:, 2]

    # Add a scatter plot for these points
    ransac_fig.add_trace(go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=2,
            color=plt.get_cmap("tab20")(i)[:3]  # Use the same color as used in your RANSAC segmentation
        )
    ))

# Update layout to have equal aspect ratio
ransac_fig.update_layout(
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis',
        aspectmode='data'
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

TypeError: __getitem__(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cuda.pybind.utility.Vector3dVector, s: slice) -> open3d.cuda.pybind.utility.Vector3dVector
    2. (self: open3d.cuda.pybind.utility.Vector3dVector, arg0: int) -> numpy.ndarray[numpy.float64[3, 1]]

Invoked with: std::vector<Eigen::Vector3d> with 347824 elements.
Use numpy.asarray() to access data., 'x'

Did you forget to `#include <pybind11/stl.h>`? Or <pybind11/complex.h>,
<pybind11/functional.h>, <pybind11/chrono.h>, etc. Some automatic
conversions are optional and require extra headers to be included
when compiling your pybind11 module.